<a href="https://colab.research.google.com/github/Yukselendincer/datasceinceproject/blob/main/VeriOptimizasyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1. Adım: Veri Setini Oluşturma
Bu bölüm, optimizasyon modelinde kullanacağımız CSV dosyalarını oluşturur.



In [ ]:
import pandas as pd
import numpy as np

# Rastgele veri üretimi için sabit değer
np.random.seed(42)

# 1. Ana Rulo Stok Verisi (Master Rolls)
master_rolls = pd.DataFrame({
    'rulo_id': [f'RULO-{i:03d}' for i in range(1, 21)],
    'genislik_mm': np.random.choice([1000, 1250, 1500], size=20),
    'kalite': np.random.choice(['DX51D', 'S235JR'], size=20)
})

# 2. Müşteri Sipariş Verisi (Orders)
orders = pd.DataFrame({
    'siparis_id': [f'SIP-{i:03d}' for i in range(1, 51)],
    'talep_genislik_mm': np.random.randint(200, 600, size=50),
    'miktar_adet': np.random.randint(1, 5, size=50),
    'kalite': np.random.choice(['DX51D', 'S235JR'], size=50)
})

print("Veri setleri başarıyla oluşturuldu.")
display(master_rolls.head())
display(orders.head())

2. Adım: Basit Bir Kesim Planı ve Fire Analizi
Bu kod, her bir siparişi rastgele bir rulo ile eşleştirip ne kadar "kenar firesi" çıktığını hesaplar. Gerçek bir optimizasyonun (MIP) ilk adımıdır.

In [ ]:
def fire_hesapla(rulo_genislik, siparis_genislik):
    if rulo_genislik < siparis_genislik:
        return None # Rulo siparişten dar olamaz
    fire = rulo_genislik % siparis_genislik
    return fire

# Örnek bir eşleştirme simülasyonu
sonuclar = []

for index, order in orders.iterrows():
    # Aynı kalitedeki uygun ruloları bul
    uygun_rulolar = master_rolls[master_rolls['kalite'] == order['kalite']]

    if not uygun_rulolar.empty:
        secilen_rulo = uygun_rulolar.iloc[0] # Basitlik için ilk uygun ruloyu seçiyoruz
        fire = fire_hesapla(secilen_rulo['genislik_mm'], order['talep_genislik_mm'])

        sonuclar.append({
            'siparis_id': order['siparis_id'],
            'rulo_id': secilen_rulo['rulo_id'],
            'rulo_genislik': secilen_rulo['genislik_mm'],
            'siparis_genislik': order['talep_genislik_mm'],
            'kenar_fire_mm': fire
        })

df_analiz = pd.DataFrame(sonuclar)
toplam_fire = df_analiz['kenar_fire_mm'].sum()

print(f"\nToplam Kenar Firesi: {toplam_fire} mm")
display(df_analiz.head())

3. Adım: Görselleştirme
Assan Bilişim mülakatında "veriyi nasıl görselleştirirsin?" sorusuna yanıt olması için:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,6))
sns.histplot(df_analiz['kenar_fire_mm'], bins=10, kde=True, color='skyblue')
plt.title('Kesim İşlemi Sonrası Fire Dağılımı')
plt.xlabel('Fire Miktarı (mm)')
plt.ylabel('İşlem Sayısı')
plt.show()

In [ ]:
# 1. Gerekli kütüphaneyi kur (Colab'da yüklü değilse)
!pip install ortools

import pandas as pd
from ortools.linear_solver import pywraplp

def create_optimization_model():
    # --- Veri Tanımlama ---
    # Bir ana rulomuz olduğunu düşünelim (1500mm genişliğinde)
    # Ve bu rulodan kesilmesi gereken farklı müşteri siparişleri:
    bin_capacity = 1500
    item_widths = [450, 450, 450, 300, 300, 550, 550, 400, 400, 250]
    num_items = len(item_widths)
    num_bins = num_items # Teorik olarak en fazla her sipariş için 1 rulo gerekir

    # --- Solver Tanımlama ---
    # SCIP, açık kaynaklı en güçlü optimizasyon çözücülerinden biridir
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # --- Değişkenler ---
    # x[i, j] = 1 eğer i. sipariş j. ruloya konulursa
    x = {}
    for i in range(num_items):
        for j in range(num_bins):
            x[i, j] = solver.IntVar(0, 1, f'x_{i}_{j}')

    # y[j] = 1 eğer j. rulo kullanılıyorsa
    y = {}
    for j in range(num_bins):
        y[j] = solver.IntVar(0, 1, f'y_{j}')

    # --- Kısıtlar ---
    # 1. Her sipariş mutlaka tam olarak 1 ruloya atanmalı
    for i in range(num_items):
        solver.Add(sum(x[i, j] for j in range(num_bins)) == 1)

    # 2. Bir rulodaki siparişlerin toplamı rulo kapasitesini aşamaz
    for j in range(num_bins):
        solver.Add(
            sum(x[i, j] * item_widths[i] for i in range(num_items)) <= y[j] * bin_capacity
        )

    # --- Amaç Fonksiyonu ---
    # Kullanılan toplam rulo sayısını minimize et (Böylece fire de minimize olur)
    solver.Minimize(solver.Sum([y[j] for j in range(num_bins)]))

    # --- Çözüm ---
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(f'Optimum Çözüm Bulundu!')
        total_bins = 0
        for j in range(num_bins):
            if y[j].solution_value() == 1:
                total_bins += 1
                items_in_bin = [item_widths[i] for i in range(num_items) if x[i, j].solution_value() == 1]
                print(f'Rulo {total_bins}: {items_in_bin} (Toplam: {sum(items_in_bin)}mm, Fire: {bin_capacity - sum(items_in_bin)}mm)')
        print(f'\nKullanılan Toplam Rulo Sayısı: {total_bins}')
    else:
        print('Çözüm bulunamadı.')

create_optimization_model()

In [ ]:
import matplotlib.pyplot as plt

def gorsellestir(sonuclar):
    fig, ax = plt.subplots(figsize=(12, 4))
    colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#BC8F8F']

    for i, rulo in enumerate(sonuclar):
        sol_kenar = 0
        for parca in rulo:
            ax.barh(f"Rulo {i+1}", parca, left=sol_kenar, height=0.6,
                   edgecolor='black', color=colors[i % len(colors)])
            ax.text(sol_kenar + parca/2, i, f"{parca}mm", va='center', ha='center', fontweight='bold')
            sol_kenar += parca

        # Fireyi göster
        fire = 1500 - sol_kenar
        if fire > 0:
            ax.barh(f"Rulo {i+1}", fire, left=sol_kenar, height=0.6,
                   color='grey', alpha=0.3, hatch='//')
            ax.text(sol_kenar + fire/2, i, f"Fire: {fire}mm", va='center', ha='center', fontsize=8)

    plt.xlim(0, 1550)
    plt.title("Çelik Rulo Kesim Optimizasyonu Sonuçları")
    plt.xlabel("Genişlik (mm)")
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.show()

# Manuel veri girişi (Görselleştirmek için)
ornek_sonuclar = [[450, 450, 450], [300, 300, 550, 250], [550, 400, 400]]
gorsellestir(ornek_sonuclar)